# Оцінка точності та налаштування гіперпараметрів моделі

### 1. Методи оцінки точності моделі

In [ ]:
#%pip install scikit-learn pandas

In [5]:

# Імпорт необхідних бібліотек
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

from sklearn import datasets


# Завантаження набору даних Iris
iris = datasets.load_iris()


In [6]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

:Number of Instances: 150 (50 in each of three classes)
:Number of Attributes: 4 numeric, predictive attributes and the class
:Attribute Information:
    - sepal length in cm
    - sepal width in cm
    - petal length in cm
    - petal width in cm
    - class:
            - Iris-Setosa
            - Iris-Versicolour
            - Iris-Virginica

:Summary Statistics:

============== ==== ==== ======= ===== ====================
                Min  Max   Mean    SD   Class Correlation
============== ==== ==== ======= ===== ====================
sepal length:   4.3  7.9   5.84   0.83    0.7826
sepal width:    2.0  4.4   3.05   0.43   -0.4194
petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
============== ==== ==== ======= ===== ====================

:Missing Attribute Values: None
:Class Distribution: 33.3% for each of 3 classes.
:Cr

In [8]:

# Перетворення даних у формат DataFrame для зручності роботи
iris_df = pd.DataFrame(
    data=iris.data,
    columns=iris.feature_names
)


In [10]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
dtypes: float64(4)
memory usage: 4.8 KB


In [11]:
iris_df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [12]:
iris_df.head(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2


In [13]:

# Додавання колонок з мітками класів
iris_df['species'] = iris.target
iris_df['species'] = iris_df['species'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

# Виведення кількох перших рядків для перевірки
print(iris_df.head())


   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

  species  
0  setosa  
1  setosa  
2  setosa  
3  setosa  
4  setosa  


Результат
Набір даних iris містить:

Вхідні ознаки (features):
Довжина чашолистка (sepal length)
Ширина чашолистка (sepal width)
Довжина пелюстки (petal length)
Ширина пелюстки (petal width)
Цільову змінну (target): Класи квітів:
0 - Setosa
1 - Versicolor
2 - Virginica
Це готовий набір, який добре підходить для задач класифікації. Ви можете з ним працювати без необхідності створювати власні локальні набори даних.

In [21]:


data = iris_df

# Підготовка даних
X = data.drop('species', axis=1)
y = data['species']

# Розділення на тренувальні та тестові вибірки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Побудова базової моделі
model = RandomForestClassifier(random_state=42, n_estimators=5)
model.fit(X_train, y_train)

# Оцінка точності базової моделі
y_pred = model.predict(X_test)
base_accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred, zero_division=0)
print("Base Accuracy:", base_accuracy)
print("Classification Report:", classification_rep)

# Перехресна перевірка
cv_scores = cross_val_score(model, X, y, cv=2)
cv_accuracy = cv_scores.mean()
print("Cross-Validation Accuracy:", cv_accuracy)


Base Accuracy: 0.9666666666666667
Classification Report:               precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       0.90      1.00      0.95         9
   virginica       1.00      0.91      0.95        11

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30

Cross-Validation Accuracy: 0.9466666666666667


### 2. Методи налаштування гіперпараметрів моделі

In [20]:

# Налаштування гіперпараметрів
param_grid = {
    'n_estimators': [5, 10, 50, 100, 150, 200],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid, cv=2, scoring='accuracy')

grid_search.fit(X_train, y_train)

# Найкращі параметри та модель
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Оцінка моделі з оптимальними параметрами
y_pred_optimized = best_model.predict(X_test)
optimized_accuracy = accuracy_score(y_test, y_pred_optimized)
print("Best Parameters:", best_params)
print("Optimized Accuracy:", optimized_accuracy)


Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 10}
Optimized Accuracy: 1.0
